<a href="https://colab.research.google.com/github/SherbyRobotics/pyro/blob/colab/examples/notebooks/pendulum_with_dp_and_custom_cost_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### DP for a pendulum swing-up

This page shows a quick demo of how DP (dynamic programming) can be used for finding global optimal control policy for a pendulum, with the option to easily modify the cost function to see how it influences the solution.

**Importing Librairies**

This page uses the toolbox *pyro*.

In [ ]:
!git clone -b dev-alex https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')

In [ ]:
import pyro
import numpy as np
import matplotlib.pyplot as plt

from IPython import display
!apt install ffmpeg

from pyro.dynamic  import pendulum
from pyro.planning import discretizer
from pyro.analysis import costfunction
from pyro.planning import valueiteration

**Defining a dynamic system model**

Here we load a already defined class including all the dynamic equations and we define the domain (for states x and torque u) for which we will generate a controller.

In [ ]:
sys  = pendulum.SinglePendulum()

# State and control input domain
sys.u_ub[0] = +5 # Max torque
sys.u_lb[0] = -5 # Min torque

sys.x_ub[0] = +6 # Max angle
sys.x_lb[0] = -6 # Min angle
sys.x_ub[1] = +6 # Max velocity
sys.x_lb[1] = -6 # Min velocity

**Defining the cost function**

Here we can define a cost function or the type:

$J = \int_{0}^{t_f}  g( x , u , t ) dt + h(x_f,t_f)$

In [ ]:
# Cost Function
class CustomCostFunction( costfunction.CostFunction ):
    """ 
    J = int( g(x,u,y,t) * dt ) + h( x(T) , T )
    """
    
    ############################
    def __init__(self):
        
        super().__init__()

        self.INF = 1E3
        self.EPS = 1E-1

        # Target state
        self.x_target = np.array([ 0, 0])

        # Quadratic cost weights
        self.Q = np.diag( np.ones(2)  )
        self.R = np.diag( np.ones(1)  )
        
        # Optionnal zone of zero cost if ||dx|| < EPS 
        self.ontarget_check = False
    
    
    #############################
    def g(self, x, u, y, t):
        """ Quadratic additive cost """
            
        # Delta values with respect to target state
        dx = x - self.x_target
        
        dJ = ( np.dot( dx.T , np.dot(  self.Q , dx ) ) +
               np.dot(  u.T , np.dot(  self.R ,  u ) ) )
        
        # Set cost to zero if on target
        if self.ontarget_check:
            if ( np.linalg.norm( dx ) < self.EPS ):
                dJ = 0
        
        return dJ

    #############################
    def h(self, x , t = 0):
        """ Final cost function with zero value """
        
        return 0

Here we define the parameters used by the cost function

In [ ]:
cf = CustomCostFunction()
cf.x_target = np.array([ -3.14 , 0 ]) # target (upright position)
cf.Q[0,0] = 1
cf.Q[1,1] = 1
cf.R[0,0] = 1

sys.cost_function = cf

print('Cost function parameters\n-----------------')
print('Target state:\n',cf.x_target)
print('Q:\n',cf.Q)
print('R:\n',cf.R)

**Synthetizing the "optimal" controller**

*VI controller*

Here we use a library function that: \\
1) Discretize the domain of the state and control inputs, by default the 2D state-space is discretized into a 101 x 101 grid, the torque is dicretized into 11 discrete level, and the time step is 0.05 sec. \\
2) Use the value-iteraton (default is 300 iterations) to compute optimal cost to go and control actions based on the previously defined cost function g(x,u,t) \\
3) Generate a continuous control law by interpolating in the computed discrete solution

In [ ]:
x_grid             = (101,101)
u_grid             = (11,1)
dt                 = 0.05
vi_iteration_steps = 200


vi = valueiteration.ValueIteration_2D( discretizer.GridDynamicSystem( sys , x_grid, u_grid, dt ) , cf )

vi.initialize()
vi.compute_steps( vi_iteration_steps ) 
vi.assign_interpol_controller()

The following figure illustrate the computed optimal cost-to-go J* for every starting state. Note that we saturate the maximum J* to plot here to better show the range of interest.

In [ ]:
max_cost_to_plot = 200

vi.plot_cost2go(max_cost_to_plot)

**Showing the computed control laws**

The next lines generate two figures showing a map illustrating the computed optimal torque to use as a function of the two system states: \\
$τ=f(θ,\dotθ)$

In [ ]:
vi.ctl.plot_control_law(sys=sys, n=100)

**Simulations**

Here we show both control law in action, with a trajectory starting at the state $[\theta=0,\dot\theta=0]$.

In [ ]:
x0 = np.array([ 0 ,0])  # initial state
tf = 10                 # simulation time

In [ ]:
cl_sys_vi      =   vi.ctl + sys 
cl_sys_vi.x0   = x0
cl_sys_vi.compute_trajectory( tf )
cl_sys_vi.plot_trajectory('xu')

**Animation of the simulations**

Here the following function generates an animation of the computed trajectory.

In [ ]:
video_vi = cl_sys_vi.generate_simulation_html_video()
html_vi  = display.HTML(video_vi)
display.display(html_vi)

**Phase-plane trajectory**

Here the same trajectory is shown on the phase-plane of the pendulum. Here the vector field illustrate the natural dynamics allong which the pendulum would evolve naturally if no torque are applied on the system.

In [ ]:
cl_sys_vi.plot_phase_plane_trajectory_closed_loop()

**Performance**

Here the performance, in terms of the defined cost-function $J = \int g(x,u,t) dt$, is shown. Note, $dJ = \frac{dJ}{dt} = g(x,u,t) $ is the increment of cost at each instant and $J$ is the cummulative cost.

In [ ]:
cl_sys_vi.plot_trajectory('j')